In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120) # display options for output
torch.set_grad_enabled(True)

In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.1.0
0.3.0


In [3]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [4]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        
        
        # (2) hidden conv layer
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size = 2, stride = 2)
        
        # (3) hidden conv layer
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size = 2, stride = 2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        
        t = F.relu(self.fc1(t))
        
        # (5) hidden linear layer
        
        t = F.relu(self.fc2(t))
        
        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t,dim = 1)
        
        return t

In [5]:
train_set = torchvision.datasets.FashionMNIST(
    root= './data/FashionMNIST'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [6]:
network = Network()

In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

In [8]:
optimizer = optim.Adam(network.parameters(), lr = 0.01)

# Training with all batches: A single epoch

In [15]:
for batch in train_loader: # Get Batch
    images, labels = batch
    total_loss = 0
    total_correct = 0
    
    preds = network(images) # pass Batch
    loss = F.cross_entropy(preds, labels) # Calculate Loss
    
    optimizer.zero_grad()
    loss.backward() # calculate Gradients
    optimizer.step() # Update Weights
    
    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)
    
print("epoch:", 0, "total_correct:", total_correct, "loss:", total_loss)

epoch: 0 total_correct: 90 loss: 0.2736070156097412


In [16]:
total_correct / len(train_set)

0.0015

#  Training with multiple epochs: The complete training loop

In [11]:
for epoch in range(5):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader: # Get Batch
        images, labels = batch
    
        preds = network(images) # pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss
    
        optimizer.zero_grad()
        loss.backward() # calculate Gradients
        optimizer.step() # Update Weights
    
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
    
    print("epoch:", epoch, "total_correct:", total_correct, "loss:", total_loss)

epoch: 0 total_correct: 51309 loss: 234.35811164975166
epoch: 1 total_correct: 51865 loss: 216.8443481773138
epoch: 2 total_correct: 52333 loss: 205.9732463657856
epoch: 3 total_correct: 52472 loss: 202.71935203671455
epoch: 4 total_correct: 52630 loss: 198.10968287289143


In [12]:
total_correct / len(train_set)

0.8771666666666667